## Предсказание оттока пользователей банка

В данной работе мы рассмотрим типовую задачу **предсказания оттока** пользователей (*Churn Prediction*). Будем использовать [датасет банковских пользователей](https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers).

Датасет имеет следующие поля:

Поле | Значение
-----|---------
CLIENTNUM | Уникальные идентификатор клиента
Attrition_Flag | Признак того, что клиент покинул банк
Customer_Age | Возраст
Gender | Пол
Dependent_count | Число зависимых членов семьи
Education_Level | Уровень образования
Marital_Status | Семейный статус
Income_Category | Уровень годового дохода
Card_Category | Тип кредитной карты
Months_on_book | Время сотрудничества с банком (мес)
Total_Relationship_Count | Общее число продуктов у клиента
Months_Inactive_12_mon | Число неактивных месяцев за последний год
Contacts_Count_12_mon | Число контактов за последний год
Credit_Limit | Кредитный лимит на карте
Total_Revolving_Bal | Общий активный баланс на карте
Avg_Open_To_Buy | Разница между кредитным лимитом по карте и текущим балансом, среднее за последний год 
Total_Amt_Chng_Q4_Q1 | Изменение суммы транзакций за год (Q4 на Q1)
Total_Trans_Amt | Сумма транзакций за год
Total_Trans_Ct | Общее число транзакций за год
Total_Ct_Chng_Q4_Q1 | Изменение количества транзакций за год (Q4 на Q1)
Avg_Utilization_Ratio | Объем средств клиента делённый на кредитный лимит (в %)

Требуется предсказать вероятность оттока **Attrition_Flag** для конкретного клиента.


In [2]:
import pandas as pd
df = pd.read_csv("http://www.soshnikov.com/temp/data/BankChurnersProcessed.zip")
df

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,...,2,3,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462
10123,710638233,Attrited Customer,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,...,2,3,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511
10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,...,3,4,5409.0,0,5409.0,0.819,10291,60,0.818,0.000
10125,717406983,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,...,3,3,5281.0,0,5281.0,0.535,8395,62,0.722,0.000


### Этап 1: Предварительный анализ данных

* Посмотрите, какой процент данных соответствует оттекающим клиентам, а какой - стабильным. Насколько такая задача классификации является сбалансированной?

> Несбалансированность датасета означает, что нам необходимо смотреть на precision и recall в качестве метрик, поскольку высокая точность не будет означать, что получившаяся модель делает хорошие предсказания.

* Изучите данные подробнее и посмотрите, есть ли в данных пропуски. Если да - решите, что с ними делать.
* Постройте графики средних значений по всем показателям отдельно для оттекающих и остающихся клиентов и сделайте вывод, какие параметры больше всего влияют на отток.
* Подумайте, какие смысловые атрибуты имеет смысл добавить в датасет. Например, средний размер транзакции за год (`Total_Trans_Amt`/`Total_Trans_Ct`).
* Для того, чтобы посмотреть влияние возраста на отток клиентов, можно построить гистограмму возрастов отдельно для оттекающих и не оттекающих клиентов и посмотреть, есть ли разница. То же самое можно проделать для других числовых признаков.



### Этап 2: Векторизация

* Разбейте данные на обучающую и тестовую выборки
* Посмотрите, какие параметры являются категориальными, и для каждого из параметров определите, является ли он номинальным или порядковым.
* Подумайте, какие параметры можно исключить из модели. Например, после вычисления среднего размера транзакции за год, можно ли исключить общий объем и количество транзакций?
* Подумайте, как использовать корреляцию между столбцами для того, чтобы понять, какие признаки можно исключить.
* Используйте `ColumnTransformer` для векторизации всех порядковых и числовых значений и получения векторизованного датасета.


### Этап 3: Обучение модели

* Обучите несколько моделей машинного обучения (логистическая регрессия, дерево решений, лес решений, SVM) и сравните полученные метрики на тестовой выборке: accuracy, precision, recall
* По возможности проинтерпретируйте полученные модели и сравните результаты с предварительным анализом. Действительно ли те признаки, которые визуально оказывали влияние на результат, учитываются моделью в первую очередь?


### Этап 4: CatBoost

* [Опционально] Посмотрите на библиотеку [CatBoost](https://catboost.ai/) от Яндекс и попробуйте использовать её для решения задачи. Сравните полученные метрики. CatBoost известна тем, что хорошо работает с категориальными данными.
